### Consume an API with Python

In [2]:
import requests

### The GET Request

In [3]:
response = requests.get('https://api.github.com')
response.status_code

200

In [ ]:
# we can use response as a boolean. 
# 200, 204, 304,... gives True
# 400, 401, ... gives False

# The __bool__ method has been written

In [4]:
bool(response)  # 200, 204, 304,...

True

In [5]:
# we can use raise for status to handle exceptions in our code

try:
    response = requests.get("https://api.github.com")
    # If the response was successful, no Exception will be raised
    response.raise_for_status()
except HTTPError as http_err:
    print(f'HTTP error occurred: {http_err}')  # Python 3.6
except Exception as err:
    print(f'Other error occurred: {err}')  # Python 3.6
else:
    print('Success!')

Success!


In [12]:
# content
type(response.content)  # content access to raw bytes from json

bytes

In [13]:
# content
type(response.text)  # this uses the default renderer

str

### Headers

In [14]:
# response.json()  # json.loads(response.text)

In [15]:
# response.headers
# response.headers.keys()

### Query String Parameters

In [16]:
import requests

# Search GitHub's repositories for requests
response = requests.get(
    'https://api.github.com/search/repositories',
    params={'q': 'requests+language:python'},
)

# Inspect some attributes of the `requests` repository
json_response = response.json()
response.status_code

200

### Headers

In [60]:
import requests

response = requests.get(
    'https://api.github.com/search/repositories',
    params={'q': 'requests+language:python'},
    headers={'Accept': 'application/vnd.github.v3.text-match+json'},
)
 # look for jsopn content

In [61]:
requests.post('https://httpbin.org/post', data={'key':'value'})
requests.put('https://httpbin.org/put', data={'key':'value'})
requests.delete('https://httpbin.org/delete')
requests.head('https://httpbin.org/get')
requests.patch('https://httpbin.org/patch', data={'key':'value'})
requests.options('https://httpbin.org/get')

<Response [200]>

In [62]:
# POST, PUT and PATCH Differences

### Inspecting yopur requests

In [ ]:
response = requests.post('https://httpbin.org/post', json={'key':'value'})
response.request.headers['Content-Type']

response.request.url

response.request.body

b'{"key": "value"}'

### Authentication

In [ ]:
from getpass import getpass
requests.get('https://api.github.com/user', auth=('username', getpass()))

# is the same
from requests.auth import HTTPBasicAuth
from getpass import getpass
requests.get(
    'https://api.github.com/user',
    auth=HTTPBasicAuth('username', getpass())
)

In [ ]:
import requests
from requests.auth import AuthBase

class TokenAuth(AuthBase):
    """Implements a custom authentication scheme."""

    def __init__(self, token):
        self.token = token

    def __call__(self, r):
        """Attach an API token to a custom auth header."""
        r.headers['X-TokenAuth'] = f'{self.token}'  # Python 3.6+
        return r


requests.get('https://httpbin.org/get', auth=TokenAuth('12345abcde-token'))

### SSL Verification

In [ ]:
requests.get('https://api.github.com', verify=False)

### Performance

In [ ]:
requests.get('https://api.github.com', timeout=3.05)

In [ ]:
# If the request establishes a connection within 2 seconds and receives data within 5 seconds o

In [ ]:
requests.get('https://api.github.com', timeout=(2, 5))

In [ ]:
import requests
from requests.exceptions import Timeout

try:
    response = requests.get('https://api.github.com', timeout=1)
except Timeout:
    print('The request timed out')
else:
    print('The request did not time out')

In [ ]:
# sessions

In [ ]:
import requests
from getpass import getpass

# By using a context manager, you can ensure the resources used by
# the session will be released after use
with requests.Session() as session:
    session.auth = ('username', getpass())

    # Instead of requests.get(), you'll use session.get()
    response = session.get('https://api.github.com/user')

# You can inspect the response just like you did before
print(response.headers)
print(response.json())

In [ ]:
# max retries

In [ ]:
import requests
from requests.adapters import HTTPAdapter
from requests.exceptions import ConnectionError

github_adapter = HTTPAdapter(max_retries=3)

session = requests.Session()

# Use `github_adapter` for all requests to endpoints that start with this URL
session.mount('https://api.github.com', github_adapter)

try:
    session.get('https://api.github.com')
except ConnectionError as ce:
    print(ce)